In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import math 

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
dl=.005
lam0 = np.arange(1,10+dl/2,dl)
lam1 = np.hstack([np.arange(-1.5,-1+dl/2,dl),np.arange(1,10+dl/2,dl)])
lam2 = np.hstack([np.arange(-10,-1+dl/2,dl),np.arange(1,10+dl/2,dl)])
#lam3 = np.hstack([np.arange(-10,-1+dl/2,dl),np.arange(1,10+dl/2,dl)])

lams = [lam0,lam1,lam2]

params = [(1,.05),(1,.05),(1,.05)]

In [4]:
A,C = 0,1
K = 61

error_FA = np.full((3,K),np.nan)
error_OR = np.full((3,K),np.nan)
error_CG2 = np.full((3,K//2+1),np.nan)
    
for trial_idx in range(3):
    
    a,c = params[trial_idx]
    Λ = lams[trial_idx].astype(np.float64)
    
    A_mat = np.diag(Λ)
    n = len(Λ)
    b_vec = np.ones(n,dtype=np.double)
    b_vec /= np.linalg.norm(b_vec)

    M = lambda x: (A*x**2+C)
    N = lambda x: (a*x**2+c)
    f = lambda x: M(x)/N(x)
    
    fAb = f(Λ)*b_vec
    
    Q,(a_,b_) = lanczos_reorth(A_mat,b_vec,K,reorth=K)
    Q2,(a2_,b2_) = lanczos_reorth(A_mat**2,b_vec,K,reorth=K)

    for i in range(1,K):
        
        # Lanczos-FA
        # solve via eigenvalue because breakdown in factorization
        T = np.diag(a_[:i]) + np.diag(b_[:i-1],1) + np.diag(b_[:i-1],-1)
        lank_FA = Q[:,:i]@np.linalg.solve(a*T@T+c*np.eye(i),(A*T@T+C*np.eye(i))[:,0])
        error_FA[trial_idx][i] = np.linalg.norm(N(Λ)**.5*(fAb - lank_FA))

        
        # Lancos-OR-lm
        lan_lm= streaming_banded_rational(n,i,(A,0,C),(a,0,c))
        for j in range(i):
            lan_lm.read_stream(Q[:,j],a_[j],b_[j])
        lan_lm.finish_up()      
            
        error_OR[trial_idx][i] = np.linalg.norm(N(Λ)**.5*(fAb - lan_lm()))
        
    for i in range(1,K//2+1):
        
        # msCG
        T2 = np.diag(a2_[:i]) + np.diag(b2_[:i-1],1) + np.diag(b2_[:i-1],-1)        
        CG2k = Q2[:,:i]@np.linalg.solve(a*T2+c*np.eye(i),(A*T2+C*np.eye(i))[:,0])
        error_CG2[trial_idx][i] = np.linalg.norm(N(Λ)**.5*(fAb - CG2k))

In [5]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

axs = []
axs.append(fig.add_axes([left, bottom, width3, height]))
axs.append(fig.add_axes([left+hoffset3, bottom, width3, height]))
axs.append(fig.add_axes([left+2*hoffset3, bottom, width3, height]))

for trial_idx in range(3):
    axs[trial_idx].plot(np.arange(K),error_FA[trial_idx]/np.linalg.norm(N(A_mat)**.5*b_vec)\
                        ,**line_styles['l2'],label='lanczos-FA')
    axs[trial_idx].plot(np.arange(0,K+1,2),error_CG2[trial_idx]/np.linalg.norm(N(A_mat)**.5*b_vec)\
                        ,**line_styles['l3'],label='msCG')
    axs[trial_idx].plot(np.arange(K),error_OR[trial_idx]/np.linalg.norm(N(A_mat)**.5*b_vec)\
                        ,**line_styles['l1'],label='lanczos-OR')
    
    axs[trial_idx].set_yscale('log')
    axs[trial_idx].set_ylim(1e-14,1e2)
    axs[trial_idx].set_xlabel('number of matvecs: $k$')

    axs[trial_idx].grid(True,linestyle=':',linewidth=.5)

axs[1].set_yticklabels('')
axs[2].set_yticklabels('')

plt.savefig(f'imgs/ch6_lanczos_msCG_squared.pdf')